# Data Aggregation and Group Operation

## Description

### GitHub連結

[GitHub連結](https://github.com/wesm/pydata-book)

### Required Dataset

[tips](https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv)  
[stock_px_2](https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/stock_px_2.csv)

## Required Packages

In [1]:
import numpy as np
import pandas as pd

## 10.1 GroupBy Mechanics

In [2]:
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5),
})
df

,key1,key2,data1,data2
0,a,one,1.381116,0.002347
1,a,two,-0.679526,-0.630039
2,b,one,-0.446746,2.220058
3,b,two,-0.425576,1.073050
4,a,one,0.471149,-0.190826


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.390913
b   -0.436161
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.926132
      two    -0.679526
b     one    -0.446746
      two    -0.425576
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.926132,-0.679526
b,-0.446746,-0.425576


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.679526
            2006   -0.446746
Ohio        2005    0.477770
            2006    0.471149
Name: data1, dtype: float64

In [8]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.390913,-0.272839
b,-0.436161,1.646554


In [9]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.926132 -0.094239
     two  -0.679526 -0.630039
b    one  -0.446746  2.220058
     two  -0.425576  1.073050

In [10]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.381116  0.002347
1    a  two -0.679526 -0.630039
4    a  one  0.471149 -0.190826
b
  key1 key2     data1     data2
2    b  one -0.446746  2.220058
3    b  two -0.425576  1.073050


In [12]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.381116  0.002347
4    a  one  0.471149 -0.190826
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.679526 -0.630039
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.446746  2.220058
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.425576  1.07305


In [13]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.446746,2.220058
3,b,two,-0.425576,1.073050


In [14]:
list(df.groupby('key1')

SyntaxError: unexpected EOF while parsing (<ipython-input-14-1c62a70ace40>, line 1)

In [ ]:
df.dtypes

In [ ]:
grouped = df.groupby(df.dtypes, axis=1)

In [ ]:
for dtype, group in grouped:
    print(dtype)
    print(group)

### Selecting a Column or Subset of Columns

In [ ]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [ ]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [ ]:
df.groupby(['key1', 'key2'])[['data2']].mean()

In [ ]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [ ]:
s_grouped.mean()

### Grouping with Dicts and Series

In [ ]:
people = pd.DataFrame(
    np.random.randn(5, 5),
    columns=['a', 'b', 'c', 'd', 'e'],
    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis']
)

In [ ]:
people.iloc[2:3, [1, 2]] = np.nan

In [ ]:
people

In [ ]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f': 'orange'
}

In [ ]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

In [ ]:
map_series = pd.Series(mapping)
map_series

In [ ]:
people.groupby(map_series, axis=1).count()

### Grouping with Functions

In [ ]:
people.groupby(len).sum()

In [ ]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

### Grouping by Index Levels

In [ ]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

In [ ]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

In [ ]:
hier_df.groupby(level='cty', axis=1).count()

## 10.2 Data Aggregation

##### Optimized groupby methods

| Function name | Description |
| - | - |
| count | Number of non-NA values in the group |
| sum | Sum of non-NA values |
| mean | Mean of non-NA values |
| median | Mean of non-NA values |
| std, var | Arithmetic median of non-NA values |
| min, max | Unbiased (n – 1 denominator) standard deviation and variance Minimum and maximum of non-NA values |
| prod | roduct of non-NA values |
| first, last | First and last non-NA values |

In [ ]:
df

In [ ]:
grouped = df.groupby('key1')

In [ ]:
grouped['data1'].quantile(0.9)

In [ ]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [ ]:
grouped.agg(peak_to_peak)

In [ ]:
grouped.describe()

### Column-Wise and Multiple Function Application

In [ ]:
tips = pd.read_csv('src/csv/tips.csv')

In [ ]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [ ]:
tips[:6]

In [ ]:
grouped = tips.groupby(['day', 'smoker'])

In [ ]:
grouped_pct = grouped['tip_pct']

In [ ]:
grouped_pct.agg('mean')

In [ ]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [ ]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [ ]:
functions = ['count', 'mean', 'max']

In [ ]:
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

In [ ]:
result['tip_pct']

In [ ]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [ ]:
grouped[['tip_pct', 'total_bill']].agg(ftuples)

In [ ]:
grouped.agg({
    'tip': np.max,
    'size': 'sum',
})

In [ ]:
grouped.agg({
    'tip_pct': ['min', 'max', 'mean', 'std'],
    'size': 'sum',
})

### Returning Aggregated Data Without Row Indexes

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

## 10.3 Apply: General split-apply-combine

In [ ]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [ ]:
top(tips, n=6)

In [ ]:
tips.groupby('smoker').apply(top)

In [ ]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

In [ ]:
result.unstack('smoker')

In [ ]:
f = lambda x: x.describe()
grouped.apply(f)

### Suppressing the Group Keys

In [ ]:
tips.groupby('smoker', group_keys=False).apply(top)

### Quantile and Bucket Analysis

In [ ]:
frame = pd.DataFrame({
    'data1': np.random.randn(1000),
    'data2': np.random.randn(1000)
})

In [ ]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

In [ ]:
def get_stats(group):
    return {
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'mean': group.mean()
    }

In [ ]:
grouped = frame.data2.groupby(quartiles)

In [ ]:
grouped.apply(get_stats).unstack()

In [ ]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [ ]:
grouped = frame.data2.groupby(grouping)

In [ ]:
grouped.apply(get_stats).unstack()

### Example: Filling Missing Values with Group-Specific Values

In [ ]:
s = pd.Series(np.random.randn(6))

In [ ]:
s[::2] = np.nan
s

In [ ]:
s.fillna(s.mean())

In [ ]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [ ]:
group_key = ['East'] * 4 + ['West'] * 4

In [ ]:
data = pd.Series(np.random.randn(8), index=states)
data

In [ ]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [ ]:
data.groupby(group_key).mean()

In [ ]:
fill_mean = lambda g: g.fillna(g.mean())

In [ ]:
data.groupby(group_key).apply(fill_mean)

In [ ]:
fill_values = {'East': 0.5, 'West': -1}

In [ ]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [ ]:
data.groupby(group_key).apply(fill_func)

### Example: Random Sampling and Permutation

In [ ]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) +  [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [ ]:
deck[:13]

In [ ]:
def draw(deck, n=5):
    return deck.sample(n)

In [ ]:
draw(deck)

In [ ]:
get_suit = lambda card: card[-1]

In [ ]:
deck.groupby(get_suit).apply(draw, n=2)

In [ ]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

### Example: Group Weighted Average and Correlation

In [ ]:
df = pd.DataFrame({
    'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
    'data': np.random.randn(8),
    'weights': np.random.randn(8)
})
df

In [ ]:
grouped = df.groupby('category')

In [ ]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [ ]:
grouped.apply(get_wavg)

In [ ]:
close_px = pd.read_csv('src/csv/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

In [ ]:
close_px[-4:]

In [ ]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [ ]:
rets = close_px.pct_change().dropna()

In [ ]:
get_year = lambda x: x.year

In [ ]:
by_year = rets.groupby(get_year)

In [ ]:
by_year.apply(spx_corr)

In [ ]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Example: Group-Wise Linear Regression

In [ ]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

## 10.4 Pivot Tables and Cross-Tabulation

In [ ]:
tips.pivot_table(index=['day', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

| Function name | Description |
| - | - |
| values | Column name or names to aggregate; by default aggregates all numeric columns |
| index | Column names or other group keys to group on the rows of the resulting pivot table |
| columns | Column names or other group keys to group on the columns of the resulting pivot table |
| aggfunc | Aggregation function or list of functions ('mean' by default); can be any function valid in a groupby context |
| fill_value | Replace missing values in result table |
| dropna | If True, do not include columns whose entries are all NA |
| margins | Add row/column subtotals and grand total (False by default) |

### Cross-Tabulations: Crosstab

In [ ]:
data